# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The model being submitted is a simple nearly identical implementation of the model referenced in the assignment. The GAP layer of ResNet-152  with its 2048 single value features is run through a trainable linear layer (plus a RELU nonlinearity) to produce a vector with the same length as the embedding vectors to be used in our decoder. The feature/context vector is passed from the encoder to the decoder for translation. The decoder is a single layer LSTM that takes the sequence of our encoded vector and the ‘correct’ image caption as its input utilizing the ‘teacher forcing’ method of seq2seq training. The output of the LSTM is passed through a linear layer which converts the LSTM word representation into a vocabulary length vector which is interpreted via  argmax or probabilistic indices sampling of the softmax version to obtain a vocabulary word. Cross-entropy loss is used to evaluate each prediction during training. Learning rates and batch sizes were experimented with to find appropriate values. The default learning rate was too high and led to sustained high loss. I suspect this is due to the fact that no batch can represent the full gradient (since no batch is likely to contain a representative of every learnable feature in the full dataset).

I tried many, many other methods, architectures and permutations thereof including inventing an attention encoder. In the end a combination of time and compute resources led me to use the simplest version of the explored methodologies.  


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The transform was not altered. With a random crop of a 256 resized image we ensure images with typical aspect ratios retain most of the central part of the image. A large number of high aspect ratio images in the training set might require a rethink of the sampling strategy. It is possible that a random flip isn't useful as the pre-trained image model was likely trained using image flips meaning the final GAP feature vector should contain the same image information regardless of image flips. In fact it may reduce quality if image captions contain references to ‘right’ and ‘left’, however I did not test this. The normalizations are set as preconditions for this particular model and should not be altered.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** Everything is trainable except the ResNet model (unless my compute resources go up a few orders of magnitude)

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**  The Adam optimizer is used if for no other reason then its frequent occurrence in recent seq2seq language translation papers/blogs.

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
import os
sys.path.append('/cocoapi/PythonAPI')
from pycocotools.coco import COCO
import math
import pickle

# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

# Import EncoderCNN and DecoderRNN.
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
# from model import AttnEncoderCNN, AttnDecoderRNN
from attentionRNN import AttnEncoderRNN, BahdanauAttnDecoderRNN

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 256 # 24 # 128  # 32 # 512           # batch size
vocab_threshold = 5 # 4         # minimum word count threshold
vocab_from_file = True      # if True, load existing vocab file
embed_size = 256 # 128          # dimensionality of image and word embeddings
hidden_size = 1024 # 512           # number of features in hidden state of the RNN decoder
num_layers = 1
num_epochs = 20 # 500             # number of training epochs
save_every = 1              # determines frequency of saving model weights
print_every = 100           # determines window for printing average loss
tag = '' # 'res152_next'               # tag for save files
log_file = 'training_log-{}.txt'.format(tag)       # name of file with saved training loss and perplexity
deep = True               # Was resnet152 used?
lr = 0.001                 # learning rate
lres = 1.0  # learning rate encoder scale
teacher_forcing_ratio = 1.0 #0.4 # How often do we use captions for training inputs
use_attn = False            # use attention model

reload = False        # reload the data_loader from a file
reload_weights = False      # reload model weights from a file
wts = 16 # weights file to load

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])


# Build data loader.
global data_loader
if reload:
    with open(os.path.join('./models', 'data_loader--{}.pkl'.format(tag)), 'rb') as f:
        data_loader = pickle.load(f)
else:
    data_loader = get_loader(transform=transform_train,
                             mode='train',
                             batch_size=batch_size,
                             vocab_threshold=vocab_threshold,
                             vocab_from_file=vocab_from_file,
                             num_workers=0)
    with open(os.path.join('./models', 'data_loader--{}.pkl'.format(tag)), 'wb') as f:
        pickle.dump(data_loader,f)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder.
# TODO #3: Specify the learnable parameters of the model.
if use_attn:
    encoder = AttnEncoderRNN(embed_size, hidden_size, deep=deep, n_layers=num_layers, dropout_p=0.25)
    print('sentence length: {}'.format(encoder.input_size))
    #decoder = AttnDecoderRNN(hidden_size, vocab_size, dropout_p=0.1, max_length=encoder.length)
    decoder = BahdanauAttnDecoderRNN(hidden_size, embed_size, vocab_size, n_layers=num_layers, dropout_p=0.1)
    encoder_params = list(filter(lambda p: p.requires_grad, encoder.parameters()))
    decoder_params = list(decoder.parameters())
    
else:
    encoder = EncoderCNN(embed_size, deep=deep)
    decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=num_layers)
    encoder_params = list(filter(lambda p: p.requires_grad, encoder.parameters()))
    decoder_params = list(decoder.parameters())

# TODO #4: Define the optimizers.
encoder_optimizer = torch.optim.Adam(encoder_params, lr=lr*lres)
decoder_optimizer = torch.optim.Adam(decoder_params, lr=lr)
#optimizer = torch.optim.SGD(params, lr=lr) # 0.00005

# Reload weights from file
if reload_weights:
    encoder_file = 'encoder-{}_{}.pkl'.format(wts,tag)
    decoder_file = 'decoder-{}_{}.pkl'.format(wts,tag)
    # Load the trained weights.
    estate = encoder.state_dict()
    encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)), strict=False)
    decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)), strict=False)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Your device:', device)
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

with open(os.path.join('./models', 'hparams--{}.pkl'.format(tag)), 'wb') as f:
    pickle.dump(    
        {'batch_size': batch_size,             # batch size
         'vocab_threshold': vocab_threshold,   # minimum word count threshold
         'vocab_from_file': vocab_from_file,   # if True, load existing vocab file
         'embed_size': embed_size,             # dimensionality of image and word embeddings
         'hidden_size': hidden_size,           # number of features in hidden state of the RNN decoder
         'num_layers': num_layers,
         'num_epochs': num_epochs,             # number of training epochs
         'save_every': save_every,             # determines frequency of saving model weights
         'transform_train': transform_train,
         'deep': deep,                         # Was resnet152 used?
         'lr': lr,
         'teacher_forcing_ratio': teacher_forcing_ratio,
         'use_attn': use_attn},                # Did we train the model with attention?
        f)
print('done.')

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import torch.utils.data as data
import numpy as np
import time

import multiprocessing as mp
import random

def get_batch(*args):
    semaphore.acquire()
    # Randomly sample a caption length, and sample indices with that length.
    #indices = args[0] 
    indices = data_loader.dataset.get_train_indices(old=False, max_len=30)  # prevent attn model from running out of GPU memory
    # Create and assign a batch sampler to retrieve a batch with the sampled indices.
    new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
    data_loader.batch_sampler.sampler = new_sampler

    # Obtain the batch.
    return next(iter(data_loader)) # images, captions

def set_batch_size(batch_size):
    data_loader.dataset.batch_size = batch_size
    data_loader.batch_sampler.batch_size = batch_size
set_batch_size(batch_size) # in case of reload with different batch size

def clean_sentence(output):
    sentence = ''
    for x in output:
        #if x == 1:
        #    break
        sentence += ' ' + data_loader.dataset.vocab.idx2word[x]
    return sentence + '   |'

# Set the total number of training steps per epoch.
cut = 1 #16 # reduce training steps
total_step = math.ceil(
    len(data_loader.dataset.caption_lengths) / (data_loader.batch_sampler.batch_size*cut))

batch_step = True # Do we reduce batch size with learning rate?
lr_step_every = 4 # 10 learning rate step down after this many epochs
lr_step_size = 0.5

# Open the training log file.
f = open(log_file, 'a')
global semaphore
semaphore = mp.Semaphore(8)
    
teach_plast = 100000.0
noTeach_plast = 100000.0
exit = False
for epoch in range(1, num_epochs+1):
    if exit:
        pool.close()
        pool.join()
        f.close()
        break
    print('batch_size: {}'.format(batch_size))
    print('learning rate {}'.format(lr))
    try:
        pool = mp.Pool()
        i_step = 0
        #indices = [(data_loader.dataset.get_train_indices()) for _ in range(1, total_step+1)]
        for images, captions in pool.imap_unordered(get_batch, range(1, total_step+1)):
            
            i_step += 1

            # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            captions = captions.to(device)

            # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()
            
            # Regularize choice of teacher forcing
            if teacher_forcing_ratio == None: 
                prob = [teach_plast, noTeach_plast]
                prob = np.array(prob) / sum(prob)
                use_teacher_forcing = np.random.choice([True, False], p=prob)
            else:
                use_teacher_forcing = True if random.random() <= teacher_forcing_ratio else False
            
            # Pass the inputs through the CNN-RNN model.
            if use_attn:
                encoder_outputs, encoder_hidden = encoder(images)
                outputs, attentions = decoder(
                    encoder_hidden, encoder_outputs, captions=captions, use_teacher_forcing=use_teacher_forcing)
                # hidden, encoder_outputs, captions=None, use_teacher_forcing=False,  max_len=None
                captions = captions[:,1:]
            else:    
                features = encoder(images)
                outputs = decoder(features, captions, use_teacher_forcing=use_teacher_forcing)           
            #print(captions)
            #exit = True
            #break
            
            # Calculate the batch loss.
            loss = criterion(outputs, captions.contiguous().view(-1))

            # Backward pass.  
            loss.backward()

            # Update the parameters in the optimizer.
            encoder_optimizer.step()
            decoder_optimizer.step()
            
            # Uptate teach stat
            pPlex = np.exp(loss.item())
            if use_teacher_forcing:
                teach_plast = pPlex
            else:
                noTeach_plast = pPlex

            # Get training statistics.
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (
                epoch, num_epochs, i_step, total_step, loss.item(), pPlex)
            stats = stats + ' Teach: {} Len: {}    '.format(use_teacher_forcing, captions.size()[1])
            
            exp = outputs.data.topk(1)[1].cpu().numpy().squeeze()[:captions.size()[1]].tolist()
            exp = clean_sentence(exp)

            # Print training statistics (on same line).
            print('\r' + stats + ' ' + exp, end="")
            sys.stdout.flush()

            # Print training statistics to file.
            # lr, batch_size, epoch, i_step, total_step, loss.item(), pPlex, use_teacher_forcing, captions.size()[1]
            txt = [str(s) for s in [lr, batch_size, epoch, i_step, total_step, loss.item(), pPlex, use_teacher_forcing, captions.size()[1]]]
            f.write(','.join(txt) + '\n')
            f.flush()

            # Print training statistics (on different line).
            if i_step % print_every == 0:
                print('\r' + stats)
            
            semaphore.release()
                
    except KeyboardInterrupt:
        pool.close()
        pool.join()
        f.close()
        
    pool.close()
    pool.join()
            
    # Save the weights and parameters.
    if epoch % save_every == 0:
        fnum = epoch
        if reload_weights:
            fnum += wts  
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-{}_{}.pkl'.format(epoch,tag)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-{}_{}.pkl'.format(epoch,tag)))
        
    if epoch % lr_step_every == 0:
        if batch_step:
            if data_loader.dataset.batch_size <= 32: 
                exit = True
                break
            batch_size = batch_size // 2 # Reduce batch size
            #batch_size = int(batch_size * 2) # Increase batch size
            set_batch_size(batch_size)
        lr = lr*lr_step_size
        encoder_optimizer = torch.optim.Adam(encoder_params, lr=lr*lres)
        decoder_optimizer = torch.optim.Adam(decoder_params, lr=lr)
        # Set the total number of training steps per epoch.
        total_step = math.ceil(len(data_loader.dataset.caption_lengths) / (data_loader.batch_sampler.batch_size*cut))        
        

# Close the training log file.
f.close()

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.